#4

In [1]:
class Rect:
    def __init__(self, x, y, width, height) -> None:
        self.x = x
        self.y = y
        self.width = width
        self.height = height

    def __eq__(self, obj) -> bool:
        return self.width == obj.width and self.height == obj.height

    def __hash__(self) -> int:
        return hash((self.width, self.height))

In [2]:
r1 = Rect(10, 5, 100, 50)
r2 = Rect(-10, 4, 100, 50)

h1, h2 = hash(r1), hash(r2)   # h1 == h2
print(h1, h2, sep='\n')

5092506917918050556
5092506917918050556


#6

In [3]:
class ShopItem:
    def __init__(self, name, weight, price) -> None:
        self.name = name
        self.weight = weight
        self.price = price

    def __hash__(self):
        return hash((self.name.lower(), self.weight, self.price))

    def __eq__(self, obj) -> bool:
        return hash(self) == hash(obj)

In [4]:
lst_in = """Системный блок: 1500 75890.56
Монитор Samsung: 2000 34000
Клавиатура: 200.44 545
Монитор Samsung: 2000 34000""".strip().split('\n')

shop_items = {}

for item in lst_in:
    curr_obj = ShopItem(item.split(':')[0], float(item.split(':')[1].split()[0]), float(item.split(':')[1].split()[1]))
    if curr_obj not in shop_items:
        shop_items[curr_obj] = [curr_obj, 1]
    else:
        shop_items[curr_obj][1] += 1

shop_items

{<__main__.ShopItem at 0x10a758340>: [<__main__.ShopItem at 0x10a758340>, 1],
 <__main__.ShopItem at 0x10a758d30>: [<__main__.ShopItem at 0x10a758d30>, 2],
 <__main__.ShopItem at 0x10a44b160>: [<__main__.ShopItem at 0x10a44b160>, 1]}

#7

In [5]:
class Record:
    ID = 1
    def __init__(self, fio, descr, old) -> None:
        self.fio = fio
        self.descr = descr
        self.old = old
        self.pk = Record.ID
        Record.ID += 1

    def __hash__(self) -> int:
        return hash((self.fio.lower(), self.old))

    def __eq__(self, obj) -> bool:
        return hash(self) == hash(obj)

class DataBase:
    def __init__(self, path) -> None:
        self.path = path
        self.dict_db = {}

    def write(self, record):
        self.dict_db.setdefault(record, [])
        self.dict_db[record].append(record)

    def read(self, pk):
        for lst in self.dict_db.values():
            for obj in lst:
                if obj.pk == pk:
                    return obj 

In [6]:
lst_in = """Балакирев С.М.; программист; 33
Кузнецов А.В.; разведчик-нелегал; 35
Суворов А.В.; полководец; 42
Иванов И.И.; фигурант всех подобных списков; 26
Балакирев С.М.; преподаватель; 37""".strip().split('\n')

db = DataBase('/asd/asd.db')

for item in lst_in:
    db.write(Record(item.split(';')[0], item.split(';')[1].split()[0], int(item.split(';')[2].split()[0])))

db.dict_db

{<__main__.Record at 0x110950d00>: [<__main__.Record at 0x110950d00>],
 <__main__.Record at 0x110950d30>: [<__main__.Record at 0x110950d30>],
 <__main__.Record at 0x110950220>: [<__main__.Record at 0x110950220>],
 <__main__.Record at 0x110950340>: [<__main__.Record at 0x110950340>],
 <__main__.Record at 0x110950160>: [<__main__.Record at 0x110950160>]}

#8

In [7]:
class BookStudy:
    def __init__(self, name, author, year) -> None:
        self.name = name
        self.author = author
        self.year = year

    def __hash__(self) -> int:
        return hash((self.name.lower(), self.author.lower()))

In [8]:
# lst_in = list(map(str.strip, sys.stdin.readlines()))

lst_in = """Python; Балакирев С.М.; 2020
Python ООП; Балакирев С.М.; 2021
Python ООП; Балакирев С.М.; 2022
Python; Балакирев С.М.; 2021""".strip().split('\n')

lst_bs = []

for item in lst_in:
    lst_bs.append(BookStudy(item.split('; ')[0], item.split('; ')[1], int(item.split('; ')[2])))

unique_books_dic = {}

for book in lst_bs:
    unique_books_dic.setdefault(hash(book), 0)
    unique_books_dic[hash(book)] += 1

unique_books = len(unique_books_dic.keys())

#9

In [9]:
class Dimensions:
    def __init__(self, a, b, c) -> None:
        if not all([type(x) in (int, float) and x > 0 for x in (a, b, c)]):
            raise ValueError("габаритные размеры должны быть положительными числами")
        self.a = a
        self.b = b
        self.c = c

    def __hash__(self) -> int:
        return hash((self.a, self.b, self.c))

In [10]:
#1 2 3; 4 5 6.78; 1 2 3; 3 1 2.5
lst_dims = [Dimensions(float(x.split()[0]), float(x.split()[1]), float(x.split()[2])) for x in input().split('; ')]
lst_dims.sort(key=hash)
[hash(x) for x in lst_dims]

[-720341356606421504,
 529344067295497451,
 529344067295497451,
 4077453281647519382]

#10

In [11]:
class FloatValue:
    @classmethod
    def check(cls, value):
        if type(value) not in (int, float) or value <= 0:
            raise ValueError("габаритные размеры должны быть положительными числами")
    
    def __set_name__(self, owner, name):
        self.name = "_" + name

    def __get__(self, instance, owner):
        return getattr(instance, self.name)

    def __set__(self, instance, value):
        self.check(value)
        setattr(instance, self.name, value)

class Triangle:
    a = FloatValue()
    b = FloatValue()
    c = FloatValue()

    def __init__(self, a, b, c) -> None:
        if not (a < b + c and b < a + c and c < a + b):
            raise ValueError("с указанными длинами нельзя образовать треугольник")
        
        self.a = a
        self.b = b
        self.c = c

    def __len__(self):
        return int(self.a + self.b + self.c)

    def __call__(self):
        p = (self.a + self.b + self.c) / 2
        return (p * (p-self.a) * (p-self.b) * (p-self.c))**0.5


